In [45]:
import pandas as pd
import numpy as np
import datetime 

Задание 1

Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [79]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [80]:
top = ratings.groupby(['userId'])['rating'].count().reset_index()
top = top.rename(columns={"rating": "user_count"})
top.head()

,userId,user_count
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100


In [81]:
top = top[(top['user_count']>=100)]
top.head()

,userId,user_count
3,4,204
4,5,100
7,8,116
14,15,1700
16,17,363


In [82]:
ratings_max = ratings[['userId','timestamp']].groupby('userId').max()
ratings_max.head()

,timestamp
userId,
1,1260759205
2,835356246
3,1298932787
4,949982274
5,1163375145


In [83]:
ratings_min = ratings[['userId','timestamp']].groupby('userId').min()
ratings_min.head()

,timestamp
userId,
1,1260759108
2,835355395
3,1298861589
4,949778714
5,1163373044


In [84]:
top = top.join(ratings_max, how='left')

In [85]:
top = top.rename(columns={"timestamp": "t_max"})

In [86]:
top = top.join(ratings_min, how='left')

In [87]:
top = top.rename(columns={"timestamp": "t_min"})
top.head()

,userId,user_count,t_max,t_min
3,4,204,1298932787,1298861589
4,5,100,949982274,949778714
7,8,116,851869291,851866703
14,15,1700,976244591,976243912
16,17,363,1178364961,1137577616


In [89]:
def ltv(row):
    return pd.to_datetime(row['t_max'],unit='s')-pd.to_datetime(row['t_min'],unit='s')

In [90]:
top['ltv'] = top.apply(ltv, axis=1)

In [91]:
top.head().sort_values('ltv', ascending=False)

,userId,user_count,t_max,t_min,ltv
16,17,363,1178364961,1137577616,472 days 01:49:05
4,5,100,949982274,949778714,2 days 08:32:40
3,4,204,1298932787,1298861589,0 days 19:46:38
7,8,116,851869291,851866703,0 days 00:43:08
14,15,1700,976244591,976243912,0 days 00:11:19


In [92]:
top['ltv'].mean()

Timedelta('199 days 10:02:16.657794')

Задание 2

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
аналогичную таблицу по типам выручки с указанием адреса клиента
Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

Дана статистика услуг перевозок клиентов компании по типам:
rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

In [93]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [94]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [95]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [96]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [103]:
clients_rev_adr = client_base.merge(rzd, on='client_id', how='left')
clients_rev_adr = clients_rev_adr.merge(air, on='client_id', how='left')
clients_rev_adr = clients_rev_adr.merge(auto, on='client_id', how='left')
clients_rev_adr.head(10)

,client_id,address,rzd_revenue,air_revenue,auto_revenue
0,111,Комсомольская 4,1093.0,NaN,NaN
1,112,Энтузиастов 8а,2810.0,NaN,NaN
2,113,Левобережная 1а,10283.0,NaN,57483.0
3,114,Мира 14,5774.0,NaN,83.0
4,115,ЗЖБИиДК 1,981.0,81.0,912.0
5,116,Строителей 18,NaN,4.0,4834.0
6,117,Панфиловская 33,NaN,13.0,98.0
7,118,Мастеркова 4,NaN,173.0,NaN


In [102]:
len(clients_rev_adr) == len(client_base)

True

In [104]:
clients_rev = clients_rev_adr.drop(columns=['address'])

In [105]:
clients_rev.head()

,client_id,rzd_revenue,air_revenue,auto_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,NaN,57483.0
3,114,5774.0,NaN,83.0
4,115,981.0,81.0,912.0


Задание 3
В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. 

Как бы вы добавили эти сведения в таблицу визитов и покупок? 

Для составления ответа можно использовать вопросы:
У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить?
Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1?
Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают.

1) создать таблицу с координатами
2) добавить в таблицу с координатами ID записи данной коррдинаты 
3) ID координаты добавить в таблицы визитов и покупок

Дополнительно из координат можно получить:
- найти группы пользователей с близкими координатами визитов или покупок друг другу
- построить ТОП категорий покупок данных групп
- карту перемещений каждого user_id и предлагать ему категрии товаров, что покупали другие user_id ближайшие к нему

In [108]:
coordinates = pd.DataFrame(
    {
        'coord_id': [1, 2, 3, 4],
        'user_id': [11, 22, 11, 77],
        'longitude': ['54.8389', '28.1189', '32.4567', '11.2345'],
        'latitude': ['84.8765', '76.9876', '45.9873', '81.3467'],
    }
)

coordinates = coordinates[['coord_id', 'user_id','longitude','latitude']]
coordinates

,coord_id,user_id,longitude,latitude
0,1,11,54.8389,84.8765
1,2,22,28.1189,76.9876
2,3,11,32.4567,45.9873
3,4,77,11.2345,81.3467


In [111]:
purchases = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 99],
        'category': ['Спорт', 'Авто', 'Дача', 'Спорт', 'Авто'],
        'coord_id': [1, 2, 'NaN', 3, 'NaN'],
    }
)

purchases = purchases[['user_id', 'category','coord_id']]
purchases

,user_id,category,coord_id
0,11,Спорт,1
1,22,Авто,2
2,55,Дача,NaN
3,11,Спорт,3
4,99,Авто,NaN


In [112]:
visits = pd.DataFrame(
    {
        'user_id': [11, 22, 55, 11, 77],
        'source': ['ad', 'yandex', 'email', 'google', 'ad'],
        'coord_id': [1, 2, 'NaN', 3, 4],
    }
)

visits = visits[['user_id', 'source','coord_id']]
visits

,user_id,source,coord_id
0,11,ad,1
1,22,yandex,2
2,55,email,NaN
3,11,google,3
4,77,ad,4
